In [36]:
import os
import dataset
import pickle

root_dir = '/chuvdatarc/ja8818/RAD/LINE/Benedetta_Franceschiello/5_OCTs_Jakub_Gwizdala/0_backup_folder_development/AMD/scans_for_segmentation'
mappings_filepath = os.path.join(root_dir, 'filename_mappings.pickle')

with open(mappings_filepath, 'rb') as mapping_file:
    mapping = pickle.load(mapping_file)

def file_id(filename):
    try:
        return int(filename[:-4])
    except ValueError:
        print(filename)
        return None

In [37]:
reverse_mapping = {}
for key, oct_scan in mapping.items():
    reverse_mapping[oct_scan.filename] = key

In [38]:
from pathlib import Path

In [39]:
segmented = []
for person in ['Astrid', 'Benedetta']:
    for group in dataset.GROUPS:
        files = os.listdir(os.path.join(root_dir, person, group))
        segmented += [(group, file_id(filename)) for filename in files]

segmented_filepaths = {mapping[key].filename for key in segmented}

with open(Path(DATASET['labelsPath']) / 'amd' / 'labelled_octs_v2.pickle', 'rb') as file:
    amd_labelled_octs_v2 = pickle.load(file)
with open(Path(DATASET['labelsPath']) / 'control' / 'labelled_octs_v2.pickle', 'rb') as file:
    control_labelled_octs_v2 = pickle.load(file)
    
all_labelled_octs = amd_labelled_octs_v2 + control_labelled_octs_v2

In [40]:
to_segment = []
to_segment_ids = []
for labelled_slice in all_labelled_octs:
    filepath = labelled_slice.oct_scan.filename
    if filepath not in segmented_filepaths:
        to_segment.append(labelled_slice)
        to_segment_ids.append(reverse_mapping.get(filepath, None))
        

import shutil
from tqdm import tqdm

In [47]:
destination = {
    'amd': os.path.join(root_dir, 'Astrid2'),
    'control': os.path.join(root_dir, 'me2')
}

v1_to_v2 = {}
filename_mappings_v2 = {}
for i, (labelled_oct, id) in enumerate(tqdm(zip(to_segment, to_segment_ids))):
    name = f'{i}_v2'
    filename = f'{name}.tif'
    if id is not None:
        v1_to_v2[f"{id[0]}/{id[1]}.meta"] = f"{id[0]}/{name}.meta"
#     try:
# #         shutil.copy2(labelled_oct.oct_scan.filename, os.path.join(destination[labelled_oct.group], filename))
#         filename_mappings_v2[name] = labelled_oct
#     except FileNotFoundError:
#         tqdm.write(f'Missing {labelled_oct.oct_scan.filename}')

458it [00:00, 264417.24it/s]


In [50]:
with open('v1_to_v2.pickle', 'wb') as file:
    pickle.dump(v1_to_v2, file)

In [48]:
lost_meta_files = []
with open('nonexistent_meta_files.pickle', 'rb') as file:
    nonexistent_meta_files = pickle.load(file)
for labelled_path, meta_filenames in nonexistent_meta_files:
    lost_meta_files += meta_filenames

In [60]:
lost_counter = 0
for lost_meta_file in lost_meta_files:
    if lost_meta_file in v1_to_v2:
        lost_counter += 1
    else:
        print('missing', lost_meta_file)
print(f'Restored {lost_counter} entries')

missing control/419.meta
missing control/420.meta
missing control/422.meta
missing control/423.meta
missing control/167.meta
missing control/260.meta
missing control/261.meta
missing control/263.meta
missing control/264.meta
missing control/266.meta
missing control/326.meta
missing control/359.meta
missing control/360.meta
missing control/362.meta
missing control/363.meta
missing control/666.meta
missing control/50.meta
missing control/51.meta
missing control/53.meta
missing control/54.meta
missing control/56.meta
Restored 75 entries


In [52]:
fm_extended = '/chuvdatarc/ja8818/RAD/LINE/Benedetta_Franceschiello/5_OCTs_Jakub_Gwizdala/0_backup_folder_development/AMD/filename_to_meta_mapping_extended.pickle'

In [53]:
with open(fm_extended, 'rb') as file:
    fm_extended = pickle.load(file)

In [59]:
counter = 0
fm_extended_and_restored = {}
for filepath, meta_filepaths in fm_extended.items():
    new_meta_filepaths = meta_filepaths.copy()
    
    # extend with restored v1 to v2 information
    for meta_filepath in meta_filepaths:
        if meta_filepath in v1_to_v2 and v1_to_v2[meta_filepath] not in meta_filepaths:
            counter += 1
            new_meta_filepaths.append(v1_to_v2[meta_filepath])
    
    fm_extended_and_restored[filepath] = new_meta_filepaths
print(f'Added {counter} entries')

Added 78 entries


In [61]:
fm_extended_and_restored_path = '/chuvdatarc/ja8818/RAD/LINE/Benedetta_Franceschiello/5_OCTs_Jakub_Gwizdala/0_backup_folder_development/AMD/filename_to_meta_mapping_extended_and_restored.pickle'
with open(fm_extended_and_restored_path, 'wb') as file:
    pickle.dump(fm_extended_and_restored, file)